In [ ]:
!pip install omegaconf
!pip install -q efficientnet
!pip install iterative-stratification

In [ ]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
user_credential = user_secrets.get_gcloud_credential()
user_secrets.set_tensorflow_credential(user_credential)

In [ ]:
from kaggle_datasets import KaggleDatasets

GCS_DS_PATH = KaggleDatasets().get_gcs_path('1tfrecordapple')
GCS_DS_PATH

In [ ]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from tensorflow import keras
import efficientnet.tfkeras as efn
# from kaggle_datasets import KaggleDatasets
from tensorflow.keras.models import Sequential
from tensorflow.keras import callbacks
import math
import tensorflow.keras.layers as L
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import schedules
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn import model_selection

from omegaconf import OmegaConf

# TPU or GPU detection
# Detect hardware, return appropriate distribution strategy

# physical_devices = tf.config.list_physical_devices('GPU')

target_cols = ["healthy", "multiple_diseases", "rust", "scab"]

conf = """
base:
  train_path: '../input/plant-pathology-2020-fgvc7/train.csv'
  test_path: "../input/plant-pathology-2020-fgvc7/test.csv"
  ss_path: "../input/plant-pathology-2020-fgvc7/sample_submission.csv"
  train_tf_path: "/train.tfrecord"
  test_tf_path: "/test.tfrecord"
  print_freq: 100
  num_workers: 4
  target_size: 4
  target_cols: ["healthy", "multiple_diseases", "rust", "scab"]
  n_fold: 4
  trn_fold: [0,1,2,3]
  train: True
  debug: False
  oof: False

dataset:
  augment: true
  cache: true
  repeat: true
  shuffle: 1024
  cache_dir: ""

split:
  # name: "MultilabelStratifiedKFold"
  name: "MultilabelStratifiedKFold"
  param: {
           "n_splits": 4,
           "shuffle": True,
           "random_state": 0
  }

model:
  model_name: "EfficientNetB7"
  size: 768  # 480
  batch_size: 64
  pretrained: true
  epochs: 20
  in_features: 2048

loss:
  name: "binary_crossentropy"
  param: {}

optimizer:
  name: "Adam"
  param: {
           "learning_rate": 1e-4,
           # "weight_decay": 1e-6,
           # "amsgrad": False
  }

scheduler:
  name: "CosineAnnealingLR"
  param: {
            "epochs_per_cycle": 10,
            "lr_max": 1e-4,
            "lr_min": 0,
            # "last_epoch": -1
  }
"""
config = OmegaConf.create(conf)

In [ ]:
def auto_select_accelerator():
    """
    Reference:
        * https://www.kaggle.com/mgornergoogle/getting-started-with-100-flowers-on-tpu
        * https://www.kaggle.com/xhlulu/ranzcr-efficientnet-tpu-training
    """
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu)
        print("Running on TPU:", tpu.master())
    except ValueError:
        strategy = tf.distribute.get_strategy()
    print(f"Running on {strategy.num_replicas_in_sync} replicas")

    return strategy


strategy = auto_select_accelerator()


def seed_everything(seed=0):
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'


seed = 2048
seed_everything(seed)
print("REPLICAS: ", strategy.num_replicas_in_sync)

# Data access
# BASE_PATH = KaggleDatasets().get_gcs_path()
train = pd.read_csv(config.base.train_path)
test = pd.read_csv(config.base.test_path)
sub = pd.read_csv(config.base.ss_path)
sub.iloc[:, 1:] = 0

In [ ]:

def decode_image(image_data, h=config.model.size, w=config.model.size):
    image = tf.image.decode_jpeg(image_data, channels=3)
    image = tf.cast(image, tf.float32) / 255.0

    image = tf.image.resize(image, [h, w])
    image = tf.reshape(image, [h, w, 3])
    return image


def build_decoder(with_labels=True):
    def read_tfrecord(example):
        # 各々のデータに対してパース
        if with_labels:
            TFREC_FORMAT = {
                'image': tf.io.FixedLenFeature([], tf.string),
                config.base.target_cols[0]: tf.io.FixedLenFeature([], tf.int64),
                config.base.target_cols[1]: tf.io.FixedLenFeature([], tf.int64),
                config.base.target_cols[2]: tf.io.FixedLenFeature([], tf.int64),
                config.base.target_cols[3]: tf.io.FixedLenFeature([], tf.int64),
                'image_name': tf.io.FixedLenFeature([], tf.string),
            }
        else:
            TFREC_FORMAT = {
                'image': tf.io.FixedLenFeature([], tf.string),
                'image_name': tf.io.FixedLenFeature([], tf.string),
            }
        example = tf.io.parse_single_example(example, TFREC_FORMAT)
        image = decode_image(example['image'])
        if with_labels:
            targets = [example[x] for x in config.base.target_cols]
            return image, targets
        else:
            return image

    return read_tfrecord


def build_augmenter(with_labels=True):
    def augment(img):
        img = tf.image.random_flip_left_right(img)
        img = tf.image.random_flip_up_down(img)
        return img

    def augment_with_labels(img, label):
        return augment(img), label

    return augment_with_labels if with_labels else augment


def data_augment(image, label=None):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)

    if label is None:
        return image
    else:
        return image, label

    
    

def build_dataset(cfg, paths, idx, labels=None, decode_fn=None, augment_fn=None, val=False):
    if val:
        cfg.dataset.repeat = False
        cfg.dataset.shuffle = False
    else:
        cfg.dataset.repeat = True
        cfg.dataset.shuffle = True

    if cfg.dataset.cache_dir != "" and cfg.dataset.cache_dir is True:
        os.makedirs(cfg.dataset.cache_dir, exist_ok=True)

    if decode_fn is None:
        decode_fn = build_decoder()

    if augment_fn is None:
        augment_fn = build_augmenter(labels is not None)

    idx = tf.constant(idx, dtype=tf.int64)

    def is_index_in(index, rest):
        return tf.math.reduce_any(index == idx)

    def drop_index(index, rest):
        return rest

    AUTO = tf.data.experimental.AUTOTUNE

    dset = tf.data.TFRecordDataset(paths)

    dset = dset.map(decode_fn, num_parallel_calls=AUTO).enumerate()
    dset = dset.filter(is_index_in)
    dset = dset.map(drop_index)

    dset = dset.cache(cfg.dataset.cache_dir) if cfg.dataset.cache_dir else dset
    dset = dset.map(augment_fn, num_parallel_calls=AUTO) if cfg.dataset.augment else dset

    dset = dset.repeat() if cfg.dataset.repeat else dset
    dset = dset.shuffle(cfg.dataset.shuffle) if cfg.dataset.shuffle else dset

    dset = dset.batch(cfg.model.batch_size).prefetch(AUTO)
    return dset


class CosineAnnealingScheduler(callbacks.LearningRateScheduler):
    def __init__(self, epochs_per_cycle, lr_min, lr_max, verbose=0):
        super(callbacks.LearningRateScheduler, self).__init__()
        self.verbose = verbose
        self.lr_min = lr_min
        self.lr_max = lr_max
        self.epochs_per_cycle = epochs_per_cycle

    def schedule(self, epoch, lr):
        return self.lr_min + (self.lr_max - self.lr_min) *\
               (1 + math.cos(math.pi * (epoch % self.epochs_per_cycle) / self.epochs_per_cycle)) / 2

__SPLITS__ = {
    "MultilabelStratifiedKFold": MultilabelStratifiedKFold,
    # "KFold": KFold,
}

__OPTIMIZER__ = {

}

__SCHEDULERS__ = {
    "CosineAnnealingLR": CosineAnnealingScheduler
}


def get_split(cfg):
    if hasattr(model_selection, cfg.split.name):
        return model_selection.__getattribute__(cfg.split.name)(**cfg.split.param)
    elif __SPLITS__.get(cfg.split.name) is not None:
        return __SPLITS__[cfg.split.name](**cfg.split.param)
    else:
        raise NotImplementedError


def get_optimizer(cfg):
    if hasattr(tf.keras.optimizers, cfg.optimizer.name):
        return tf.keras.optimizers.__getattribute__(cfg.optimizer.name)(**cfg.optimizer.param)
    elif __OPTIMIZER__.get(cfg.optimizer.name) is not None:
        return __OPTIMIZER__[cfg.optimizer.name](**cfg.optimizer.param)
    else:
        raise NotImplementedError


def get_scheduler(cfg):
    if hasattr(schedules, cfg.scheduler.name):
        return schedules.__getattribute__(cfg.scheduler.name)(**cfg.scheduler.param)
    elif __SCHEDULERS__.get(cfg.scheduler.name) is not None:
        return __SCHEDULERS__[cfg.scheduler.name](**cfg.scheduler.param)
    else:
        raise NotImplementedError


def main(cfg):
    global train
    seed_everything(seed=cfg.base.seed)

    folds = train.copy()

    if cfg.base.debug:
        folds = folds.sample(n=100, random_state=cfg.base.seed).reset_index(drop=True)
        cfg.model.epochs = 1
    Fold = get_split(cfg)
    for n, (train_index, val_index) in enumerate(Fold.split(folds, folds[cfg.base.target_cols])):
        folds.loc[val_index, 'fold'] = int(n)
    folds['fold'] = folds['fold'].astype(int)

    oof_df = train.copy()


    for fold in range(cfg.base.n_fold):
        if fold in cfg.base.trn_fold:
            fold_pred = train_loop(cfg, folds, fold)
            sub.iloc[:, 1:] += fold_pred / cfg.base.n_fold
            # test_pred[list(cfg.base.target_cols)] += fold_pred / len(cfg.base.trn_fold)

    sub.to_csv("submit.csv", index=False)


def train_loop(cfg, folds, fold):
    global rand

    # folds["path"] = cfg.base.train_image_path + folds["image_id"].astype(str) + ".jpg"

    trn_idx = folds[folds['fold'] != fold].index
    val_idx = folds[folds['fold'] == fold].index

    train_folds = folds.loc[trn_idx].reset_index(drop=True)
    valid_folds = folds.loc[val_idx].reset_index(drop=True)

    decoder = build_decoder()
    # test_decoder = build_decoder(with_labels=Fals, target_size=(cfg.model.size, cfg.model.size))
    train_tf_path = cfg.base.train_tf_path

    train_dataset = build_dataset(
        cfg,
        train_tf_path,
        trn_idx.values,
        # augment_fn=augmenter,
        decode_fn=decoder,
        labels=train_folds[cfg.base.target_cols],
        val=False
    )

    valid_dataset = build_dataset(
        cfg,
        train_tf_path,
        val_idx.values,
        decode_fn=decoder,
        labels=valid_folds[cfg.base.target_cols],
        val=True
    )

    with strategy.scope():
        model = tf.keras.Sequential([
            efn.__getattribute__(cfg.model.model_name)(
                input_shape=(cfg.model.size, cfg.model.size, 3),
                weights='imagenet',
                include_top=False,
                drop_connect_rate=0.1),
            tf.keras.layers.GlobalAveragePooling2D(),
            tf.keras.layers.Dense(cfg.base.target_size, activation='sigmoid')
        ])
        model.compile(
            optimizer=get_optimizer(cfg),
            loss=cfg.loss.name,
            metrics=[tf.keras.metrics.AUC(multi_label=True)])
        model.summary()

    steps_per_epoch = train_folds.shape[0] // cfg.model.batch_size
    checkpoint = tf.keras.callbacks.ModelCheckpoint(
        'model.h5', save_best_only=True, monitor='val_auc', mode='max')
    lr_reducer = get_scheduler(cfg)

    history = model.fit(
        train_dataset,
        epochs=cfg.model.epochs,
        verbose=2,
        callbacks=[checkpoint, lr_reducer],
        steps_per_epoch=steps_per_epoch,
        validation_data=valid_dataset
    )

    # oof = model.predict(valid_dataset, verbose=1)

    test_tf_path = cfg.base.test_tf_path
    test_augment = build_augmenter(with_labels=False)
    test_decoder = build_decoder(with_labels=False)
    test_dataset = build_dataset(
        cfg,
        test_tf_path,
        test.index.values,
        decode_fn=test_decoder,
        augment_fn=test_augment,
        val=True
    )

    pred = model.predict(test_dataset, verbose=1)

    return pred




In [ ]:
config.base.train_tf_path = GCS_DS_PATH  + config.base.train_tf_path
config.base.test_tf_path = GCS_DS_PATH + config.base.test_tf_path

In [ ]:
config.base.train_tf_path

In [ ]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
user_credential = user_secrets.get_gcloud_credential()
user_secrets.set_tensorflow_credential(user_credential)

In [ ]:
main(config)